In [ ]:
import networkx as nx

g: nx.DiGraph = nx.cycle_graph(10, nx.DiGraph)
g.add_edge(0, 5)
nx.draw(g, pos=nx.circular_layout(g), with_labels=True)
a = nx.community.louvain_communities(g)
#for i in g.nodes:
#    print(f"[{i}]:", comm[i])



TypeError: '_AxesStack' object is not callable

<Figure size 640x480 with 0 Axes>

In [ ]:
import matplotlib.pyplot as plt
import random
print(f"検出されたコミュニティ数: {len(a)}")
for i, comm in enumerate(a):
    subG = g.subgraph(comm)  # コミュニティ部分グラフ
    pos = nx.spring_layout(subG, seed=42)

    # ノード・エッジの色（見やすいよう同系色系統で統一）
    color = plt.cm.tab20(i % 20)  # 20色パレット
    edge_colors = [color for _ in subG.edges]

    plt.figure(figsize=(6, 6))

    # エッジを先に描画（ノードの下地）
    nx.draw_networkx_edges(
        subG,
        pos,
        edge_color=edge_colors,
        alpha=0.5,
        width=1.0
    )

    # ノードを上に重ねる
    nx.draw_networkx_nodes(
        subG,
        pos,
        node_color=color,
        node_size=50
    )

    plt.title(f"Community {i} (size={len(comm)})")
    plt.axis("off")
    plt.tight_layout()
    plt.show()

#描画で見る限り有向グラフだった

In [ ]:
twitter: nx.DiGraph = nx.read_edgelist("./ego-Twitter/twitter_combined.txt.gz", create_using=nx.DiGraph)
twitter.number_of_nodes(), twitter.number_of_edges(), twitter.is_directed()

In [3]:
import networkx as nx

twitter: nx.DiGraph = nx.read_edgelist("sampled_twitter_graph.txt", create_using=nx.DiGraph)

twitter.number_of_nodes(), twitter.number_of_edges(), twitter.is_directed()

twitter_comms = nx.community.louvain_communities(twitter)

print(f"検出されたコミュニティ数: {len(twitter_comms)}")

KeyboardInterrupt: 

In [78]:
import requests
import tarfile
import os
url = "https://snap.stanford.edu/data/twitter.tar.gz"
filename = "ego-twitter/twitter.tar.gz"
data_file = "ego-twitter/twitter_combined.txt"
output_dir = "twitter_communities2"

if not os.path.exists(data_file):
    print(f"'{filename}'をダウンロードしています...")
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        with open(filename, "wb") as f:
            for chunk in r.iter_content(chunk_size=1024):
                f.write(chunk)
        print("ダウンロードが完了しました。")

        print(f"'{filename}'を展開しています...")
        with tarfile.open(filename, "r:gz") as tar:
            tar.extractall()
        print("展開が完了しました。")
    else:
        print("データのダウンロードに失敗しました。")
        exit()

twitter: nx.DiGraph = nx.read_edgelist(data_file, create_using=nx.DiGraph)
twitter.number_of_nodes(), twitter.number_of_edges(), twitter.is_directed()

(81306, 1768149, True)

In [ ]:
twitter_comms = nx.community.louvain_communities(twitter)

In [ ]:
print(f"検出されたコミュニティ数: {len(twitter_comms)}")

検出されたコミュニティ数: 82


In [ ]:
import pandas as pd
import numpy as np
# --- コミュニティごとのグループ化 ---
communities = {}
for node, community_id in twitter_comms.items():
    communities.setdefault(community_id, []).append(node)

# --- コミュニティの密度分析 ---
community_stats = []
for community_id, nodes in communities.items():
    if len(nodes) > 1:
        subgraph = twitter.subgraph(nodes)
        community_stats.append({
            "community_id": community_id,
            "num_nodes": subgraph.number_of_nodes(),
            "num_edges": subgraph.number_of_edges(),
            "density": nx.density(subgraph)
        })
df_stats = pd.DataFrame(community_stats)

In [ ]:
output_dir = "twitter_communities3"
# 密度が1.0の完全グラフ（クリック）のコミュニティを除外して、最も密なコミュニティを探します
# （完全グラフは自明な密なコミュニティのため、それ以外で興味深いものを探します）
most_dense_community = df_stats[df_stats['density'] < 1.0].sort_values(by='density', ascending=False).iloc[0]

# エッジが1つ以上あるコミュニティの中で、最も疎なコミュニティを探します
most_sparse_community = df_stats[df_stats['num_edges'] > 0].sort_values(by='density', ascending=True).iloc[0]

# 密度の中央値を計算
median_density = df_stats['density'].median()

# 中央値との差の絶対値を計算し、最も差が小さいコミュニティを特定
df_stats['median_diff'] = (df_stats['density'] - median_density).abs()
median_community = df_stats.sort_values(by='median_diff', ascending=True).iloc[0]

print("\n--- 分析結果 ---")
print(f"\n密度の中央値: {median_density:.6f}")

print("\n✅ 最も密なコミュニティ（完全グラフを除く）:")
print(most_dense_community.to_string())

print("\n\n✅ 中央値に近いコミュニティ:")
print(median_community[['community_id', 'num_nodes', 'num_edges', 'density']].to_string())

print("\n\n✅ 最も疎なコミュニティ（エッジが1つ以上あるもの）:")
print(most_sparse_community.to_string())
print("\n---------------\n")

print(f"\nコミュニティごとの隣接行列を'{output_dir}'に保存しています...")

In [ ]:
# 各コミュニティについてループ処理を行います
for community_id, nodes in communities.items():
    # サブグラフを作成します
    subgraph = G.subgraph(nodes)
    
    # サブグラフの隣接行列をNumPy配列として取得します
    # ノードの順序を固定するためにnodelistを指定します
    adj_matrix = nx.to_numpy_array(subgraph, nodelist=sorted(subgraph.nodes()))
    
    # ご提示のコードに合わせてデータ型を整数に変換します
    adj_matrix = adj_matrix.astype(np.int32)
    
    # NumPy配列をpandas DataFrameに変換します
    # indexとcolumnsにノードIDリストを指定することで、JSONがより分かりやすくなります
    node_list = sorted(subgraph.nodes())
    df_adj = pd.DataFrame(adj_matrix, index=node_list, columns=node_list)
    
    # 保存するファイル名を定義します
    output_filename = os.path.join(output_dir, f"community_{community_id}_adj.txt")
    
    # DataFrameをJSONファイルとして保存します
    df_adj.to_json(output_filename, orient="index")

print("JSONファイルの保存が完了しました。")